# Random Forest with Dataset ***Bank Marketing***
- https://archive.ics.uci.edu/dataset/222/bank+marketing

# Import library

In [ ]:
%pip install ucimlrepo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Fetch and read data

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
bank_marketing = fetch_ucirepo(id=222)

# data (as pandas dataframes)
X = bank_marketing.data.features
y = bank_marketing.data.targets

df = pd.concat([X, y], axis=1)

In [ ]:
# Xem phân phối của biến target
print(y.value_counts())

y  
no     39922
yes     5289
Name: count, dtype: int64


> Target lệch nhiều về 'no', có thể khi dự đoán có thể không dự đoán tốt ở TH 'yes'

In [ ]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day_of_week,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,may,261,1,-1,0,NaN,no
1,44,technician,single,secondary,no,29,yes,no,NaN,5,may,151,1,-1,0,NaN,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,NaN,5,may,76,1,-1,0,NaN,no
3,47,blue-collar,married,NaN,no,1506,yes,no,NaN,5,may,92,1,-1,0,NaN,no
4,33,NaN,single,NaN,no,1,no,no,NaN,5,may,198,1,-1,0,NaN,no


# Describe dataset

In [ ]:
df.shape

(45211, 17)

In [ ]:
df.isnull().sum().sort_values(ascending=False).rename('Missing Values').to_frame()

,Missing Values
poutcome,36959
contact,13020
education,1857
job,288
month,0
previous,0
pdays,0
campaign,0
duration,0
age,0


In [ ]:
# Check duplicated rows
print("Duplicated rows: ", df.duplicated().sum())

Duplicated rows:  0


# Data preprocessing

In [ ]:
# Delete 3 features have many missing values: 'poutcome', 'education', 'contact'
df.drop(columns=['poutcome', 'education', 'contact'], inplace=True)

df.head()

,age,job,marital,default,balance,housing,loan,day_of_week,month,duration,campaign,pdays,previous,y
0,58,management,married,no,2143,yes,no,5,may,261,1,-1,0,no
1,44,technician,single,no,29,yes,no,5,may,151,1,-1,0,no
2,33,entrepreneur,married,no,2,yes,yes,5,may,76,1,-1,0,no
3,47,blue-collar,married,no,1506,yes,no,5,may,92,1,-1,0,no
4,33,NaN,single,no,1,no,no,5,may,198,1,-1,0,no


In [ ]:
df.isnull().sum().sort_values(ascending=False).rename('Missing Values').to_frame()

,Missing Values
job,288
age,0
marital,0
default,0
balance,0
housing,0
loan,0
day_of_week,0
month,0
duration,0


In [ ]:
# Delete rows have missing values: 'job'
df.dropna(subset=['job'], inplace=True)

In [ ]:
df.isnull().sum().sort_values(ascending=False).rename('Missing Values').to_frame()

,Missing Values
age,0
job,0
marital,0
default,0
balance,0
housing,0
loan,0
day_of_week,0
month,0
duration,0


In [ ]:
# Encode
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

encode_feature_cols = ['job', 'marital', 'default', 'housing', 'loan', 'month', 'y']


# Encode features
for col in encode_feature_cols:
  df[col] = le.fit_transform(df[col])

In [ ]:
df.head()

,age,job,marital,default,balance,housing,loan,day_of_week,month,duration,campaign,pdays,previous,y
0,58,4,1,0,2143,1,0,5,8,261,1,-1,0,0
1,44,9,2,0,29,1,0,5,8,151,1,-1,0,0
2,33,2,1,0,2,1,1,5,8,76,1,-1,0,0
3,47,1,1,0,1506,1,0,5,8,92,1,-1,0,0
5,35,4,1,0,231,1,0,5,8,139,1,-1,0,0


# Split data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['y']), df['y'], test_size=0.33, random_state=42)

# Fitting and Evaluating the Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.97      0.95     13094
           1       0.63      0.36      0.46      1731

    accuracy                           0.90     14825
   macro avg       0.77      0.67      0.70     14825
weighted avg       0.89      0.90      0.89     14825



*Đánh giá*
1. Precision (Độ chính xác)
- Đối với lớp "no", độ chính xác là 0.92. Điều này có nghĩa là khi mô hình dự đoán "no", nó đúng 92% trong các trường hợp.
- Đối với lớp "yes", độ chính xác là 0.63, tức là khi mô hình dự đoán "yes", nó đúng 63% trong các trường hợp.
2. Recall (Độ nhạy)
- Đối với lớp "no", độ nhạy là 0.97, cho thấy mô hình nắm bắt được 97% các trường hợp "no" thực tế.
- Đối với lớp "yes", độ nhạy là 0.36, nghĩa là nó chỉ nắm bắt được 36% các trường hợp "yes" thực tế. Độ nhạy thấp này cho thấy mô hình gặp khó khăn trong việc nhận diện các trường hợp "yes".
3. F1-Score
- Đối với lớp "no", F1-score là 0.95, cho thấy sự cân bằng cao giữa độ chính xác và độ nhạy.
- Đối với lớp "yes", F1-score là 0.51, cho thấy hiệu suất thấp hơn. F1-score thấp ở đây phản ánh sự khó khăn của mô hình trong việc nhận diện chính xác các trường hợp "yes".
4. Support
- Lớp "no" có 13,094 mẫu, trong khi lớp "yes" có 1,731 mẫu. Sự mất cân đối này cho thấy rằng các trường hợp "no" phổ biến hơn, điều này có thể ảnh hưởng đến hiệu suất của mô hình trên lớp "yes".
5. Các chỉ số tổng thể
- Accuracy: Độ chính xác tổng thể của mô hình là 0.90, có nghĩa là mô hình phân loại đúng 90% trong tất cả các trường hợp.
- Macro Average: Trung bình không trọng số tính trung bình không có trọng số của độ chính xác, độ nhạy và F1-score, coi cả hai lớp như nhau mà không tính đến sự mất cân đối. Ở đây, các giá trị trung bình không trọng số là:
  - Độ chính xác: 0.77
  - Độ nhạy: 0.67
  - F1-score: 0.70
- Weighted Average: Trung bình có trọng số tính trung bình của độ chính xác, độ nhạy và F1-score, có trọng số theo số lượng mẫu (support) của từng lớp. Điều này mang lại trọng số cao hơn cho lớp "no" do tần suất của nó cao hơn. Ở đây, các giá trị trung bình có trọng số là:
  - Độ chính xác: 0.89
  - Độ nhạy: 0.90
  - F1-score: 0.89